In [0]:
# Delete both checkpoint AND schema location
dbutils.fs.rm("/Volumes/workspace/damg7370/datastore/Food_Inspections/checkpoint/Chicago_Bronze_Layer", recurse=True)
dbutils.fs.rm("/Volumes/workspace/damg7370/datastore/Food_Inspections/schema/Chicago_Bronze_Layer", recurse=True)

In [0]:
import dlt
from pyspark.sql.functions import current_timestamp, col

@dlt.table(
    table_properties={
        "delta.columnMapping.mode": "name"
    }
)
def chicago_bronze():
    return (
        spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "csv")
            .option("header", "true")
            .option("inferSchema", "true")
            .load("/Volumes/workspace/damg7370/datastore/Food_Inspections")
            .filter(col("_metadata.file_name") == "Chicago_Bronze_Layer.csv")  # Filter for only Chicago file
            .select(
                "*",
                current_timestamp().alias("load_dt"),
                col("_metadata.file_path").alias("source_file_path"),
                col("_metadata.file_name").alias("source_file_name")
            )
    )